# Auswertung
Wir haben jetzt eine klassische Segmentierung erstellt, ein Noise2Void Modell trainiert um zu entrauschen, sowie das Stardist Modell für *instance segmentation* und zwei U-Nets für *semantic segmentation* erstellt. Nun geht es darum, die Methoden miteinander zu vergleichen, Vor- und Nachteile zu finden. Dabei gibt es natürlich qualitative Unterschiede, die man direkt mit dem Auge schon entdeckt hat. Wir sollten diese Unterschiede aber auch objektiv mit quantitativen Methoden untersuchen. *Recall*, *Precision* und *F1-Score* sind Größen, die dafür in Frage kommen.

Damit die Modelle vorurteilsfrei evaluiert werden können, verwenden wir nun das Test Set.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import scipy
import skimage
from skimage import restoration
from skimage import filters
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm
from skimage.measure import label, regionprops
from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.matching import matching_dataset
from stardist.models import StarDist2D

from n2v.models import N2VConfig, N2V
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator


from csbdeep.utils import plot_history

#np.random.seed(6)
lbl_cmap = random_label_cmap()
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
# We import all our dependencies.

## Qualitativer Vergleich Entrauschen
Wir wollen nun einen Vergleich der verschiedenen Entrauschungsmethoden vornehmen. Hier verwenden wir ein Bild aus dem Validierungsset. Wir zeigen das Input Bild, und verschiedene klassische Denoising Methoden zum Vergleich. Warum können wir mit unseren Daten keinen quantitativen Vergleich vornehmen, bspw. mittels Peak Signal to Noise Ratio (PSNR)?

### Daten

In [ ]:
datagen = N2V_DataGenerator()
imgs = datagen.load_imgs_from_directory(directory = "/extdata/readonly/f-prak-v15/v-cholera-biofilm")
# wir verwenden nur die Validation Bilder
imgs = imgs[len(imgs)//4:]
# und wählen eines der Bilder zufällig aus
raw = img = imgs[np.random.randint(0, len(imgs))][0, ..., 0]

In [ ]:
# denoise mit Gauß
gauss = filters.gaussian(raw.astype(float), 0.8)
# denoise mit non-local mean
# https://scikit-image.org/docs/dev/auto_examples/filters/plot_nonlocal_means.html
sigma = restoration.estimate_sigma(raw, multichannel=False)
nlm = restoration.denoise_nl_means(
    raw, h=0.8*sigma, sigma=sigma, fast_mode=True,
    patch_size=10, patch_distance=13,
)
# Zuletzt denoising mit stardist
model = N2V(config=None, name="n2v_biofilm", basedir="models")
denoise_n2v = model.predict(raw, axes='YX')

Nun vergleichen wir die entrauschten Bilder und einen Aussschnitt aus den Bildern.

In [ ]:
def plot_effect(imgs, labels, cmaps=None):
    cols = len(imgs)
    rows = len(imgs[0])
    if cmaps is None:
        cmaps = [None,] * rows
    fig, axes = plt.subplots(cols, rows, figsize=[16, cols*5])
    for i in range(cols):
        for j in range(rows):
            try:
                ax = axes[i, j]
            except IndexError:
                ax = axes[j]
            if not imgs[i][j] is None:
                ax.imshow(imgs[i][j], cmap=cmaps[j])
                ax.axis("off")
                ax.set_title(labels[i][j])
            else:
                ax.remove()

slx = slice(450,550)
sly = slice(450,550)
out = [raw, gauss, nlm, denoise_n2v]

plot_effect(
    [
        out,
        [o[sly,slx] for o in out],
    ],
    [
        ["raw", "gauss filter size=0.8", "nlm sigma={:1.0f}".format(sigma), "n2v"],
        [""]*4
    ]
)

## Aufgabe 6-1
Verbessert die obige Abbildung, sodass für beide Zeilen eine Scalebar vorhanden ist. Außerdem sollte erkenntlich sein, von wo der Ausschnit in der unteren Zeile stammt. Vergleich in der Auswertung die Bilder qualitativ.

## GPU leeren
Damit wir im folgenden die GPU mit anderen Modellen beladen können, befreien wir die GPU nun vom N2V Modell.

In [ ]:
del model
tf.keras.backend.clear_session()

## Quantitativer Vergleich Segmentierung
Kopiert aus dem zweiten Notebook eure Segmentier Funktion.

In [ ]:
def segment(x):
    gauss = filters.gaussian(x.astype(float), 2)
    white_tophat = scipy.ndimage.morphology.white_tophat(gauss, 10)
    thresh = filters.threshold_otsu(white_tophat)
    binary = white_tophat > thresh
    labelmap = label(binary)
    return labelmap

### Daten laden

In [ ]:
X_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/test/input/*.tif'))
Y_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/test/labels/*.tif'))
X = [x.astype(int) for x in list(map(imread, X_glob))]
Y = [x.astype(int) for x in list(map(imread, Y_glob))]

In [ ]:
model_stardist = StarDist2D(None, name='mystardist-1', basedir='04_stardist/models')
# ladet auch eure anderen UNet models

In [ ]:
Y_klassisch = [segment(x) for x in X]
Y_stardist = [
    model_stardist.predict_instances(normalize(x, 1, 99.8, axis=(0,1))) for x in X
]
Y_stardist_label = [x[0] for x in Y_stardist]
# predictions für eure Models sollen hier hin


In [ ]:
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats_tat = [matching_dataset(Y, Y, thresh=t, show_progress=False) for t in tqdm(taus)]
stats_klassisch = [matching_dataset(Y, Y_klassisch, thresh=t, show_progress=False) for t in tqdm(taus)]
stats_stardist = [matching_dataset(Y, Y_stardist_label, thresh=t, show_progress=False) for t in tqdm(taus)]

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(15,21))

def plot_stats(stats, ax1, ax2, title):
    for m in ('precision', 'recall', 'accuracy', 'f1', 'mean_true_score'):
        ax1.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
    ax1.set_xlabel(r'IoU threshold $\tau$')
    ax1.set_ylabel('Metric value')
    ax1.grid()
    ax1.legend()
    ax1.set_title(title)

    for m in ('fp', 'tp', 'fn'):
        ax2.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
    ax2.set_xlabel(r'IoU threshold $\tau$')
    ax2.set_ylabel('Number #')
    ax2.grid()
    ax2.legend();
plot_stats(stats_klassisch, axes[0, 0], axes[0, 1], "klassische Segmentierung")
plot_stats(stats_stardist, axes[1, 0], axes[1, 1], "stardist Segmentierung")
#'plot_stats(stats_tat, axes[2, 0], axes[2, 1], "wenn alles komplett richtig ist")

Vergleicht auch die Anzahl der Zellen, die segmentiert worden sind. Gibt es Unterschiede? Wie lassen sich die Unterschiede erklären. Welche weiteren Möglichkeiten gibt es zusammenhängende Zellen in separate Zellen zu separieren.

## Qualitativer Vergleich
Nun betrachten wir für die verschiedenen Methoden einen Ausschnitt gemeinsam mit der Segmentiermaske.

In [ ]:
slx = slice(500, 700)
sly = slice(200, 400)
i = 0
inp = X[i]
seg = [y[i] for y in [Y_klassisch, Y_stardist_label, Y]]
titles = ["klassisch", "stardist", "GT"]

fig, axes = plt.subplots(2, len(seg), figsize=(21,21))
for ax, s, title in zip(axes[0,:], seg, titles):
    ax.imshow(inp, cmap="gray")
    ax.imshow(s, cmap=lbl_cmap, alpha=0.2)
    ax.set_title(title)
    ax.axis("off")
    scalebar = ScaleBar(1/6, "µm")
    ax.add_artist(scalebar)
for ax, s, title in zip(axes[1,:], seg, titles):
    ax.imshow(inp[sly, slx], cmap="gray")
    ax.imshow(s[sly, slx], cmap=lbl_cmap, alpha=0.2)
    ax.set_title(title)
    ax.axis("off")
    scalebar = ScaleBar(1/6, "µm")
    ax.add_artist(scalebar)